ALGORITHM:

step1: Open the URL "https://money.cnn.com/data/hotstocks/" and read the contents of the web page

step2: Convert the contents to BeautifulSoup object

step3: Using find_next_sibling() function of BeautifulSoup, locate the tables with Most Actives, Gainers and Losers data

step4: Iterate through the rows of each table and get the stock names and ticker symbols.

step5: For collecting the data about each stock, read the data from the website "https://finance.yahoo.com/quote/" using the respective ticker symbol

step6: Locate the following data for each stock: Previous Close, Open Price, Volume, Market Cap, using find() method of BeautifulSoup

step7: Insert the data collected form each stock into the dataframe called "stocks"

step8: After collecting the data of all the stocks, export the dataframe to excel file with name "stocks_data.csv"

step9: Display the stocks of Most Active, Gainers and Losers category to the user

step10: Probe the user to enter a ticker symbol of a stock

step12: If the ticker symbol is not present in the stocks dataframe, print "entered ticker symbol does not exist"

step13: If the ticker symbol exists, get the index and print it contents to the user

Note: user inputs are case sensitive.

The cell below extracts the content from "https://money.cnn.com/data/hotstocks/" and "https://finance.yahoo.com/quote/" and the required data is scraped from the two websites and stored in stocks dataframe. This dataframe is exported to csv file

In [46]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup,Tag
import ssl
import csv
import pandas as pd 
  
# column name list
col_names =  ['type','ticker_symbol','stock_name','Open','PrevClose','Volume','MarketCap']
#col_names =  ['stock_name','ticker_symbol','type','Open','PrevClose','Volume','MarketCap'] 
stocks = pd.DataFrame(columns = col_names) 

# step1: Open the URL "https://money.cnn.com/data/hotstocks/" and read the contents of the web page
url = "https://money.cnn.com/data/hotstocks/" 
html = urllib.request.urlopen(url).read()

# step2: Convert the contents to BeautifulSoup object
soup = BeautifulSoup(html,'lxml')

docList = ['Most Actives','Gainers','Losers']

count = 0

for lists in docList:  
    header = soup.find('h3', text = lists)
    
    #step3: Using find_next_sibling() function of BeautifulSoup, locate the tables with Most Actives, Gainers and Losers data    
    table = header.find_next_sibling('table')
    rows = table.find_all('tr')
    
    # step4: Iterate through the rows of each table and get the stock names and ticker symbols.
    for i in range(1,len(rows)):
        
        # step5: For collecting the data about each stock, read the data from the website "https://finance.yahoo.com/quote/" using the respective ticker symbol
        url2 = "https://finance.yahoo.com/quote/"+rows[i].td.a.string+"?"
        html2 = urllib.request.urlopen(url2).read()
        soup2 = BeautifulSoup(html2,'lxml')
        
        # step6: Locate the following data for each stock: Previous Close, Open Price, Volume, Market Cap, using find() method of BeautifulSoup
        previous = soup2.find('td', text="Previous Close")
        previousString = previous.find_next_sibling('td').span.string

        Open = soup2.find('td', text="Open")
        OpenString = Open.find_next_sibling('td').span.string 

        Volume = soup2.find('td', text="Volume")
        VolumeString = Volume.find_next_sibling('td').span.string

        MarketCap = soup2.find('td', text="Market Cap")
        MarketCapString = MarketCap.find_next_sibling('td').span.string

        soup2 = BeautifulSoup(html2,'lxml')
        
        # step7: Insert the data collected form each stock into the dataframe called "stocks"
        stocks.loc[count] = [lists,rows[i].td.a.string,rows[i].td.span.string,OpenString,previousString,VolumeString,MarketCapString]
#        stocks.loc[count] = [rows[i].td.span.string,rows[i].td.a.string,lists,OpenString,previousString,VolumeString,MarketCapString]
        count += 1

# step8: After collecting the data of all the stocks, export the dataframe to excel file with name "stocks_data.csv"        
stocks.to_csv('stocks_data.csv')        


The cell below displays the companies under Most Actives, Gainers and Losers categories. It takes the ticker symbol of a stock as input from the user and displays the information about that stock

In [47]:
count1 = 0
count2 = 0

print("This is a program to scrape data from the https://money.cnn.com/data/hotstocks/  for a class project.  \n")
print("Which stock are you interested in: \n")

print("Most Actives:")

# step9: Display the stocks of Most Active, Gainers and Losers category to the user
for index,value in enumerate(stocks.index):
    if stocks.loc[index]['type'] == 'Most Actives':
        print(stocks.loc[index]['ticker_symbol'],stocks.loc[index]['stock_name'])
    elif stocks.loc[index]['type'] == 'Gainers':
        if count1 == 0:
            print("\n")
            print("Gainers:")
            print(stocks.loc[index]['ticker_symbol'],stocks.loc[index]['stock_name'])
            count1 += 1
        else:
            print(stocks.loc[index]['ticker_symbol'],stocks.loc[index]['stock_name'])
    elif stocks.loc[index]['type'] == 'Losers':
        if count2 == 0:
            print("\n")
            print("Losers:")
            print(stocks.loc[index]['ticker_symbol'],stocks.loc[index]['stock_name'])
            count2 += 1
        else:
            print(stocks.loc[index]['ticker_symbol'],stocks.loc[index]['stock_name'])
            

print("\n")

# step10: Probe the user to enter a ticker symbol of a stock
userInput = input("User inputs:")

tickerSymIndex = stocks.ticker_symbol[stocks.ticker_symbol == userInput].index.tolist()

# step13: If the ticker symbol exists, get the index and print it contents to the user
if len(tickerSymIndex) > 0:
    print("The data for {} {} is the following:\n".format(stocks.loc[tickerSymIndex[0]]['ticker_symbol'],stocks.loc[tickerSymIndex[0]]['stock_name']))
    print("{} {}".format(stocks.loc[tickerSymIndex[0]]['ticker_symbol'],stocks.loc[tickerSymIndex[0]]['stock_name']))
    print("OPEN: {}".format(stocks.loc[tickerSymIndex[0]]['Open']))
    print("PREV CLOSE: {}".format(stocks.loc[tickerSymIndex[0]]['PrevClose']))
    print("VOLUME: {}".format(stocks.loc[tickerSymIndex[0]]['Volume']))
    print("MARKET CAP: {}".format(stocks.loc[tickerSymIndex[0]]['MarketCap']))
else:
    
    # step12: If the ticker symbol is not present in the stocks dataframe, print "entered ticker symbol does not exist"
    print("entered ticker symbol does not exist")

This is a program to scrape data from the https://money.cnn.com/data/hotstocks/  for a class project.  

Which stock are you interested in: 

Most Actives:
GE General Electric Co
F Ford Motor Co
PFE Pfizer Inc
CCL Carnival Corp
BAC Bank of America Corp
T AT&T Inc
BA Boeing Co
MRO Marathon Oil Corp
OXY Occidental Petroleum Corp
WFC Wells Fargo & Co


Gainers:
LB L Brands Inc
TYL Tyler Technologies Inc
PHM Pultegroup Inc
DHI D.R. Horton Inc
NEM Newmont Corporation
LEN Lennar Corp
EFX Equifax Inc
WHR Whirlpool Corp
FBHS Fortune Brands Home & Security Inc
WELL Welltower Inc


Losers:
LUMN Centurylink Inc
SPG Simon Property Group Inc
FTI TechnipFMC PLC
DVN Devon Energy Corp
HFC HollyFrontier Corp
WMB Williams Companies Inc
NLSN Nielsen Holdings PLC
ALK Alaska Air Group Inc
NOV National Oilwell Varco Inc
WRB W. R. Berkley Corp


User inputs:WMB
The data for WMB Williams Companies Inc is the following:

WMB Williams Companies Inc
OPEN: 22.30
PREV CLOSE: 22.49
VOLUME: 11,251,704
MARKET CAP: 26